In [4]:
import pandas as pd
import numpy as np

In [5]:
from sklearn.impute import SimpleImputer, KNNImputer

In [43]:
# Read the CSV file into a Pandas DataFrame
df = pd.read_csv("C:\\Users\\Ju\\Downloads\\titanic\\train.csv")

# Print the DataFrame
print(df)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ... 

In [ ]:
#Première description du jeu de données

In [33]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [31]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
PassengerId,891.0,446.000000,257.353842,1.00,223.5000,446.0000,668.5,891.0000
Survived,891.0,0.383838,0.486592,0.00,0.0000,0.0000,1.0,1.0000
Pclass,891.0,2.308642,0.836071,1.00,2.0000,3.0000,3.0,3.0000
Age,714.0,29.699118,14.526497,0.42,20.1250,28.0000,38.0,80.0000
SibSp,891.0,0.523008,1.102743,0.00,0.0000,0.0000,1.0,8.0000
Parch,891.0,0.381594,0.806057,0.00,0.0000,0.0000,0.0,6.0000
Fare,891.0,32.204208,49.693429,0.00,7.9104,14.4542,31.0,512.3292


In [30]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
#Premier preprocessing : 

In [47]:
#Delete rows with Embarked information missing
df = df[df['Embarked'].notnull()]

In [50]:
def impute_data(df):
  # Create imputers for different strategies
  imputer_num = SimpleImputer(strategy="mean") # Impute numerical features with mean

    #NB: Pour cat, vu que 2/3 de cabin manquent vaut mieux faire dropna?
    #NB2: Pour num, y'a que age donc...
  # Separate numerical and categorical columns
  num_cols = df.select_dtypes(include=[np.number])

  # Impute numerical and categorical features separately
  df[num_cols.columns] = imputer_num.fit_transform(num_cols)

  return df

In [51]:
data_imputed = impute_data(df.copy())

In [ ]:
#A noter, il nous manque encore une technique d'imputation pour 'Cabin'

In [ ]:
#Ensuite, on cherche à numériser des variables catégorielles

In [18]:
categorical_cols = data_imputed.select_dtypes(include=[object])
print("Columns with object dtype (potentially categorical):")
print(categorical_cols.columns.tolist())

Columns with object dtype (potentially categorical):
['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']


In [26]:
data_imputed["Sex"] = data_imputed["Sex"].apply(lambda x: 0 if x == "male" else 1)

In [28]:
print(data_imputed["Sex"])  # Print the transformed sex column

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    1
889    0
890    0
Name: Sex, Length: 891, dtype: int64


In [30]:
from sklearn.preprocessing import LabelEncoder

# Assuming data_imputed is your DataFrame

# Create a LabelEncoder object
le = LabelEncoder()

# Encode the "Embarked" column
data_imputed["Embarked"] = le.fit_transform(data_imputed["Embarked"])

print(data_imputed["Embarked"])  # Print the encoded "Embarked" column

0      2
1      0
2      2
3      2
4      2
      ..
886    2
887    2
888    2
889    0
890    1
Name: Embarked, Length: 891, dtype: int32


In [ ]:
#On a numérisé embarked et sex. Pour Name on peut tester le titre, et Ticket et Cabin je sais pas encore quoi faire.
#Disons que cabin, on peut le drop: en effet, si plus de la moitié des observations manquent, on risque d'avoir des estimateurs très biaisés si l'on modélise avec cabin.

In [15]:
df_modified = data_imputed.drop("Cabin", axis=1)  # axis=1 specifies dropping a column

In [20]:
#Title feature: the lambda function splits the Name string at the comma
#and again at the period following, to isolate passenger title.

df_modified['Title'] = df_modified['Name'].map(lambda x: x.split(', ')[1].split('.')[0])
df_modified['Title'].unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
       'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'the Countess',
       'Jonkheer'], dtype=object)

In [28]:
#Marche pas pour l'instant
import pandas as pd
#One-hot encoding titles
#(feeds in alphabetically sorted list of unique titles)
df_modified[sorted(list(df_modified['Title'].unique()))] = pd.get_dummies(df_modified['Title'])